In [21]:
import spacy
import json
import os
from pathlib import Path
import random
from spacy.training import Example
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from spacy.training import offsets_to_biluo_tags, biluo_tags_to_offsets

In [17]:
BASE_DIR = Path.cwd().resolve().parents[2]
DATA_DIR = os.path.join(BASE_DIR, 'data', 'cleaned_standard_annotated')
FOLDER_NAMES = ['Python', 'ML', 'Android', 'DevOps', 'dotNET', 'FrontEnd', 'Golang', 'Java', 'macOS', 'Node', 'PHP']
# SAVE_DIR = os.path.join(BASE_DIR, 'data', 'texts_ner_cleaned_standard')
TAGS_PATH = os.path.join(BASE_DIR, 'data', 'annotated_manual', 'tags.json')

In [16]:
# Loading annotated NER data
def load_annotated_data(base_dir):
    data = []
    for folder in FOLDER_NAMES:
        folder_path = os.path.join(DATA_DIR, folder)
        if not os.path.exists(folder_path):
            continue

        for root, _, files in os.walk(folder_path):
            for filename in files:
                if filename.endswith(".json"):
                    file_path = os.path.join(root, filename)
                    with open(file_path, "r", encoding="utf-8") as f:
                        file = json.load(f)
                        text = file['text']
                        entities = [
                            (start, end, label)
                            for start, end, label in file.get("entities", [])
                        ]
                        if text.strip():
                            data.append((text, {"entities": entities}))

    print(f"Loaded {len(data)} annotated samples")
    return data

data = load_annotated_data(DATA_DIR)

Loaded 702 annotated samples


In [18]:
with open(TAGS_PATH, "r", encoding="utf-8") as f:
    tags = json.load(f)

labels = [t["name"] for t in tags]
print(f"Loaded {len(labels)} entity labels from tags.json:\n{labels}")

train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)
print(f"Train: {len(train_data)} | Test: {len(test_data)}")

nlp = spacy.blank("en") 
for pipe_name in nlp.pipe_names:
    nlp.remove_pipe(pipe_name)

ner = nlp.add_pipe("ner")

for label in labels:
    ner.add_label(label)

Loaded 10 entity labels from tags.json:
['SKILL_HARD', 'SKILL_SOFT', 'ENGLISH_LEVEL', 'DEGREE', 'EXPERIENCE_LEVEL', 'EXPERIENCE_YEARS', 'BENEFIT', 'LOCATION', 'COMPANY_NAME', 'ROLE']
Train: 561 | Test: 141


In [ ]:
def remove_overlapping_entities(entities):
    entities = sorted(entities, key=lambda x: (x[0], x[1]))
    cleaned = []
    last_end = -1

    for start, end, label in entities:
        if start < last_end:   # Overlap → skip
            continue
        cleaned.append((start, end, label))
        last_end = end

    return cleaned

cleaned_train = []
for text, ann in train_data:
    ann = ann.copy()
    ann["entities"] = remove_overlapping_entities(ann["entities"])
    cleaned_train.append((text, ann))

cleaned_test = []
for text, ann in test_data:
    ann = ann.copy()
    ann["entities"] = remove_overlapping_entities(ann["entities"])
    cleaned_test.append((text, ann))

# Replace original datasets
train_data = cleaned_train
test_data = cleaned_test

optimizer = nlp.initialize()
n_iter = 100

for i in range(n_iter):
    random.shuffle(train_data)
    losses = {}

    for text, annotations in tqdm(train_data, desc=f"Epoch {i+1}/{n_iter}"):

        doc = nlp.make_doc(text)
        example = Example.from_dict(doc, annotations)

        nlp.update([example], drop=0.2, sgd=optimizer, losses=losses)

    print(f"Losses at iteration {i+1}: {losses}")


correct, total = 0, 0

for text, ann in test_data:
    doc = nlp(text)

    pred = {(e.start_char, e.end_char, e.label_) for e in doc.ents}
    true = set(tuple(e) for e in ann["entities"])

    correct += len(pred & true)
    total += len(true)

print("Accuracy:", correct / total if total > 0 else 0)


Epoch 1/100:   0%|          | 2/561 [00:00<01:38,  5.70it/s]c:\Users\Admin\AppData\Local\Programs\Python\Python310\lib\site-packages\spacy\training\iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "Software Engineer (React+Node)
About Mindly

Mindl..." with entities "[(0, 17, 'ROLE'), (25, 29, 'SKILL_HARD'), (37, 43,...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
Epoch 1/100:   2%|▏         | 11/561 [00:01<01:05,  8.45it/s]c:\Users\Admin\AppData\Local\Programs\Python\Python310\lib\site-packages\spacy\training\iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "Senior DevOps Engineer
Leobit (leobit.com) is a fu..." with entities "[(0, 6, 'EXPERIENCE_LEVEL'), (7, 22, 'ROLE'), (23,...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligne

Losses at iteration 1: {'ner': 23077.52220586671}


Epoch 2/100: 100%|██████████| 561/561 [01:05<00:00,  8.51it/s]


Losses at iteration 2: {'ner': 12050.704921103868}


Epoch 3/100: 100%|██████████| 561/561 [01:04<00:00,  8.68it/s]


...

Losses at iteration 98: {'ner': 2088.0132327108086}


Epoch 99/100: 100%|██████████| 561/561 [01:03<00:00,  8.83it/s]


Losses at iteration 99: {'ner': 2005.4334445909176}


Epoch 100/100: 100%|██████████| 561/561 [01:04<00:00,  8.77it/s]


Losses at iteration 100: {'ner': 2111.600992172622}
Accuracy: 0.8253863134657836


In [25]:
precision = correct / total if total > 0 else 0
print(f"Baseline precision: {precision:.3f}")

Baseline precision: 0.825


In [27]:
example_text = random.choice(test_data)[0]
doc = nlp(example_text)
print("Example predictions:")
for ent in doc.ents:
    print(f"{ent.text:<30} → {ent.label_}")

Example predictions:
DevOps Engineer                → ROLE
Calaton Systems                → COMPANY_NAME
Tampa                          → SKILL_HARD
Florida                        → SKILL_HARD
USA                            → LOCATION
Ukraine                        → LOCATION
Poland                         → LOCATION
DevOps Engineer                → ROLE
Calaton Systems                → COMPANY_NAME
CI/CD                          → SKILL_HARD
Terraform                      → SKILL_HARD
Ansible                        → SKILL_HARD
4+ years                       → EXPERIENCE_YEARS
scripting                      → SKILL_HARD
Python                         → SKILL_HARD
Bash                           → SKILL_HARD
Java                           → SKILL_HARD
JavaScript                     → SKILL_HARD
Ansible                        → SKILL_HARD
Puppet                         → SKILL_HARD
Chef                           → SKILL_HARD
Linux                          → SKILL_HARD
Nginx              

In [28]:
examples = []
for text, annotations in test_data:
    doc = nlp.make_doc(text)
    examples.append(Example.from_dict(doc, annotations))

scorer = nlp.evaluate(examples)

print("Evaluation results:")
print(f"Precision: {scorer['ents_p']:.3f}")
print(f"Recall:    {scorer['ents_r']:.3f}")
print(f"F-score:   {scorer['ents_f']:.3f}")
print(f"Entities per type:\n{scorer['ents_per_type']}")

c:\Users\Admin\AppData\Local\Programs\Python\Python310\lib\site-packages\spacy\training\iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "Team Lead, Full Stack Engineering
At SPD Technolog..." with entities "[(0, 9, 'ROLE'), (11, 33, 'ROLE'), (37, 51, 'COMPA...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
c:\Users\Admin\AppData\Local\Programs\Python\Python310\lib\site-packages\spacy\training\iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "BackEnd (Node.js) Developer
Moon Active is one of ..." with entities "[(9, 16, 'SKILL_HARD'), (28, 39, 'COMPANY_NAME'), ...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
c:\Users\Admin\AppData\Local\Programs\Python\Python310

Evaluation results:
Precision: 0.829
Recall:    0.832
F-score:   0.831
Entities per type:
{'EXPERIENCE_LEVEL': {'p': 0.8624338624338624, 'r': 0.9157303370786517, 'f': 0.888283378746594}, 'SKILL_HARD': {'p': 0.8255227108868061, 'r': 0.8641509433962264, 'f': 0.8443952802359881}, 'COMPANY_NAME': {'p': 0.8880866425992779, 'r': 0.656, 'f': 0.7546012269938651}, 'LOCATION': {'p': 0.9235880398671097, 'r': 0.8714733542319749, 'f': 0.8967741935483872}, 'ENGLISH_LEVEL': {'p': 0.7746478873239436, 'r': 0.7857142857142857, 'f': 0.7801418439716311}, 'ROLE': {'p': 0.7425149700598802, 'r': 0.6595744680851063, 'f': 0.6985915492957746}, 'EXPERIENCE_YEARS': {'p': 0.9041095890410958, 'r': 0.9166666666666666, 'f': 0.9103448275862068}, 'SKILL_SOFT': {'p': 0.7264150943396226, 'r': 0.7475728155339806, 'f': 0.7368421052631577}, 'BENEFIT': {'p': 0.8033707865168539, 'r': 0.8171428571428572, 'f': 0.810198300283286}, 'DEGREE': {'p': 0.7333333333333333, 'r': 0.7857142857142857, 'f': 0.7586206896551724}}
